In [164]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [165]:
!pip install pygeohash
!pip install catboost
!pip install lightgbm
!pip install xgboost
!pip install joblib

In [166]:
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
from tqdm import tqdm
import joblib
import math

import pygeohash as pgh

from concurrent.futures import ThreadPoolExecutor

from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
from xgboost import XGBRegressor
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.linear_model import LinearRegression

from sklearn.model_selection import KFold
from sklearn.metrics import *
from sklearn.preprocessing import LabelEncoder

import gc
gc.collect()

import os

In [167]:
path  = "/content/drive/My Drive/signate/SMBC Group GREEN×DATA Challenge 2024/"
# 予測モデルを訓練するためのデータセット
train_df = pd.read_csv(path+'train.csv', index_col=0)

# 予測モデルに推論（予測)させるデータセット
test_df = pd.read_csv(path+'test.csv', index_col=0)

In [168]:
test_df.shape

(2508, 20)

In [169]:
# すべての列を表示するように設定
pd.set_option('display.max_columns', None)
train_df.columns

Index(['FacilityName', 'Latitude', 'Longitude', 'LocationAddress', 'City',
       'State', 'ZIP', 'County', 'FIPScode', 'PrimaryNAICS',
       'SecondPrimaryNAICS', 'IndustryType', 'TRI_Air_Emissions_10_in_lbs',
       'TRI_Air_Emissions_11_in_lbs', 'TRI_Air_Emissions_12_in_lbs',
       'TRI_Air_Emissions_13_in_lbs', 'GHG_Direct_Emissions_10_in_metric_tons',
       'GHG_Direct_Emissions_11_in_metric_tons',
       'GHG_Direct_Emissions_12_in_metric_tons',
       'GHG_Direct_Emissions_13_in_metric_tons',
       'GHG_Direct_Emissions_14_in_metric_tons'],
      dtype='object')

In [170]:
train_df = train_df.reset_index(drop=True)
test_df  = test_df.reset_index(drop=True)

In [171]:
# del train_df['Unnamed: 0'], test_df['Unnamed: 0']
del train_df['FacilityName'], test_df['FacilityName']
del train_df['LocationAddress'], test_df['LocationAddress']
del train_df['ZIP'], test_df['ZIP']
del train_df['IndustryType'], test_df['IndustryType']
del train_df['SecondPrimaryNAICS'], test_df['SecondPrimaryNAICS']

In [172]:

two_digit_map     = {11: 'Agriculture, Forestry, Fishing and Hunting',
                    21: 'Mining, Quarrying, and Oil and Gas Extraction',
                    22: 'Utilities',
                    23: 'Construction',
                    31: 'Manufacturing',
                    32: 'Manufacturing',
                    33: 'Manufacturing',
                    42: 'Wholesale Trade',
                    44: 'Retail Trade',
                    45: 'Retail Trade',
                    48: 'Transportation and Warehousing',
                    49: 'Transportation and Warehousing',
                    51: 'Information',
                    52: 'Finance and Insurance',
                    53: 'Real Estate and Rental and Leasing',
                    54: 'Professional, Scientific, and Technical Services',
                    55: 'Management of Companies and Enterprises',
                    56: 'Administrative and Support and Waste Management and Remediation Services',
                    61: 'Educational Services',
                    62: 'Health Care and Social Assistance',
                    71: 'Arts, Entertainment, and Recreation',
                    72: 'Accommodation and Food Services',
                    81: 'Other Services (except Public Administration)',
                    92: 'Public Administration'}

In [173]:
train_df['first_two_digit_primary_naics'] = train_df['PrimaryNAICS'].apply(lambda z: str(z)[:2]).astype(int)
test_df['first_two_digit_primary_naics']  = test_df['PrimaryNAICS'].apply(lambda z: str(z)[:2]).astype(int)

train_df['Economic_Sector']               = train_df['first_two_digit_primary_naics'].map(two_digit_map)
test_df['Economic_Sector']                = test_df['first_two_digit_primary_naics'].map(two_digit_map)

del train_df['first_two_digit_primary_naics'], test_df['first_two_digit_primary_naics']

econ_sector_train                         = train_df['Economic_Sector'].values
econ_sector_test                          = test_df['Economic_Sector'].values

In [174]:
test_df.shape

(2508, 16)

In [175]:



def haversine(lat1, lon1, lat2, lon2):
    # Convert latitude and longitude from degrees to radians
    lat1, lon1, lat2, lon2 = map(math.radians, [lat1, lon1, lat2, lon2])

    # Haversine formula
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = math.sin(dlat / 2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon / 2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))

    # Radius of Earth in kilometers (mean radius)
    R = 6371.0

    # Calculate the distance
    distance = R * c
    return distance

In [176]:
def calculate_distance(lat1, lon1, lat2, lon2):
    return haversine(lat1, lon1, lat2, lon2)

def get_nearest_distance(lat1, lon1, econ_sector_ref, neighbours=5, train_point=False):
    # Extract the required columns once to NumPy arrays
    latitudes = train_df['Latitude'].values
    longitudes = train_df['Longitude'].values
    ghg_emissions = train_df['GHG_Direct_Emissions_14_in_metric_tons'].values
    econ_sectors = train_df['Economic_Sector'].values

    # Filter NaN emissions directly
    valid_indices = ~np.isnan(ghg_emissions)

    latitudes = latitudes[valid_indices]
    longitudes = longitudes[valid_indices]
    ghg_emissions = ghg_emissions[valid_indices]
    econ_sectors = econ_sectors[valid_indices]

    # Parallelize distance calculation using ThreadPoolExecutor
    with ThreadPoolExecutor() as executor:
        distances = list(executor.map(calculate_distance,
                                     [lat1]*len(latitudes),
                                     [lon1]*len(longitudes),
                                     latitudes,
                                     longitudes))

    # Combine the results into a DataFrame
    near_df = pd.DataFrame({
        'Distance': distances,
        'GHG_emission_14': ghg_emissions,
        'Economic_Sector': econ_sectors
    })


    if train_point:
        near_df.sort_values(by='Distance', inplace=True)
        near_df = near_df.dropna()
        near_df = near_df.iloc[1:].reset_index(drop=True)
    else:
        # Sort distances by 'Distance'
        near_df.sort_values(by='Distance', inplace=True)
        near_df = near_df.dropna()

    # Filter by economic sector
    nearest_locations_econ_sector = near_df[near_df['Economic_Sector'] == econ_sector_ref]

    # Get the top N nearest for both economic sector and overall
    sub_near_econ = nearest_locations_econ_sector.head(neighbours)
    sub_nearest_locations = near_df.head(neighbours)

    # Compute weighted averages
    econ_weighted_average = (sub_near_econ['GHG_emission_14'] / np.where(sub_near_econ['Distance'] == 0, 1, sub_near_econ['Distance']) ).sum()
    near_weighted_average = (sub_nearest_locations['GHG_emission_14'] / np.where(sub_nearest_locations['Distance']==0,1,sub_nearest_locations['Distance'])).sum()

    # Compute regular averages
    econ_average = sub_near_econ['GHG_emission_14'].mean()
    near_average = sub_nearest_locations['GHG_emission_14'].mean()

    return [econ_weighted_average, econ_average, near_weighted_average, near_average]

In [177]:
neighbours = 5

# テストデータの距離計算または読み込み
file_path = f"test_distance_{neighbours}nbrs.csv"
if os.path.exists(file_path):
    print(f"{file_path} exists.")
    test_distance = pd.read_csv(file_path)
else:
    output = []
    for index in tqdm(range(test_df.shape[0])):
        lat1 = test_df.iloc[index]['Latitude']
        lon1 = test_df.iloc[index]['Longitude']
        econ_sector_ref = test_df.iloc[index]['Economic_Sector']
        x = get_nearest_distance(lat1, lon1, econ_sector_ref, neighbours=neighbours, train_point=False)
        output.append(x)

    test_distance = pd.DataFrame(output, columns=['Economy_Sector_Weighted_Avg', 'Economic_Sector_Average', 'Nearest_Weighted_Average', 'Nearest_Average'])
    test_distance.to_csv(file_path, index=False)

# トレーニングデータの距離計算または読み込み
file_path = f"train_distance_{neighbours}nbrs.csv"  # f-string を正しく使用
if os.path.exists(file_path):
    print(f"{file_path} exists.")
    train_distance = pd.read_csv(file_path)
else:
    output = []
    for index in tqdm(range(train_df.shape[0])):
        lat1 = train_df.iloc[index]['Latitude']
        lon1 = train_df.iloc[index]['Longitude']
        econ_sector_ref = train_df.iloc[index]['Economic_Sector']
        x = get_nearest_distance(lat1, lon1, econ_sector_ref, neighbours=neighbours, train_point=True)
        output.append(x)

    train_distance = pd.DataFrame(output, columns=['Economy_Sector_Weighted_Avg', 'Economic_Sector_Average', 'Nearest_Weighted_Average', 'Nearest_Average'])
    train_distance.to_csv(file_path, index=False)

train_distance = train_distance.reset_index(drop=True)
test_distance  = test_distance.reset_index(drop=True)

test_distance_5nbrs.csv exists.
train_distance_5nbrs.csv exists.


In [178]:
test_df

,Latitude,Longitude,City,State,County,FIPScode,PrimaryNAICS,TRI_Air_Emissions_10_in_lbs,TRI_Air_Emissions_11_in_lbs,TRI_Air_Emissions_12_in_lbs,TRI_Air_Emissions_13_in_lbs,GHG_Direct_Emissions_10_in_metric_tons,GHG_Direct_Emissions_11_in_metric_tons,GHG_Direct_Emissions_12_in_metric_tons,GHG_Direct_Emissions_13_in_metric_tons,Economic_Sector
0,38.033040,-97.973170,HUTCHINSON,KS,RENO,20155.0,211112,NaN,NaN,NaN,NaN,88951.390376,65803.021457,2.899329e+04,32965.487915,"Mining, Quarrying, and Oil and Gas Extraction"
1,32.316030,-108.606800,LORDSBURG,NM,HIDALGO,35023.0,486210,NaN,NaN,NaN,NaN,NaN,NaN,3.119260e+05,268668.069110,Transportation and Warehousing
2,41.251500,-78.742440,BROCKPORT,PA,JEFFERSON,42065.0,327213,28275.061385,32075.140635,26445.792090,23385.161510,84908.372871,6212.899604,3.473291e+04,39045.247387,Manufacturing
3,44.959660,-93.193410,SAINT PAUL,MN,RAMSEY,27123.0,322130,37330.205714,42503.897432,43906.102080,42958.306538,49004.603631,52.038452,1.426237e+04,20204.924986,Manufacturing
4,32.818720,-117.125800,SAN DIEGO,CA,SAN DIEGO,6073.0,333611,45310.722699,39897.800897,40203.428231,39437.099224,84959.769922,15351.359676,4.000654e+04,63935.249120,Manufacturing
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2503,38.651945,-75.914445,HURLOCK,MD,DORCHESTER,24019.0,562212,NaN,NaN,NaN,NaN,110523.066755,98954.793233,3.467008e+06,147802.356027,Administrative and Support and Waste Managemen...
2504,31.361795,-101.776403,RANKIN,TX,UPTON,48461.0,211112,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Mining, Quarrying, and Oil and Gas Extraction"
2505,41.005565,-96.154917,LOUISVILLE,NE,CASS,31025.0,327310,22956.125554,23592.402994,25069.498286,24560.172725,166520.232511,469732.826538,8.255855e+04,85775.307153,Manufacturing
2506,39.364417,-93.449361,CARROLLTON,MO,CARROLL,29033.0,325193,124460.895022,140816.449347,93118.705227,130326.946431,71773.803649,936031.894142,3.369613e+05,156640.371706,Manufacturing


In [179]:
test_distance

,Economy_Sector_Weighted_Avg,Economic_Sector_Average,Nearest_Weighted_Average,Nearest_Average
0,28019.568906,369614.866259,454847.563793,606222.120889
1,1296.575343,75372.922188,26392.933789,299096.231937
2,59857.420688,44340.597367,81533.914173,203831.976280
3,45265.635701,127292.222361,297072.418525,405635.394345
4,8838.742950,70847.235694,617665.034364,290001.288780
...,...,...,...,...
2503,46677.018970,344616.752313,45282.942220,309317.094507
2504,290186.877822,544751.143291,290186.877822,544751.143291
2505,102525.917615,658668.333532,100759.314574,598531.117832
2506,3259.275220,58047.779667,8262.408868,91773.030660


ここで、もうおかしい

In [180]:
train_df = pd.concat([train_df, train_distance], axis=1)
test_df = pd.concat([test_df, test_distance], axis=1)


In [181]:
def create_features1(df):


    # Year-over-year difference
    df['TRI_Air_Emissions_YoY_Change_11'] = df['TRI_Air_Emissions_11_in_lbs'] - df['TRI_Air_Emissions_10_in_lbs']
    df['TRI_Air_Emissions_YoY_Change_12'] = df['TRI_Air_Emissions_12_in_lbs'] - df['TRI_Air_Emissions_11_in_lbs']
    df['TRI_Air_Emissions_YoY_Change_13'] = df['TRI_Air_Emissions_13_in_lbs'] - df['TRI_Air_Emissions_12_in_lbs']

   # Year-over-year growth rate with NaN protection
    df['TRI_Air_Emissions_Growth_Rate_11'] = np.where(
        df['TRI_Air_Emissions_10_in_lbs'].notna() & (df['TRI_Air_Emissions_10_in_lbs'] != 0),
        (df['TRI_Air_Emissions_11_in_lbs'] - df['TRI_Air_Emissions_10_in_lbs']) / df['TRI_Air_Emissions_10_in_lbs'],
        np.nan
    )

    df['TRI_Air_Emissions_Growth_Rate_12'] = np.where(
        df['TRI_Air_Emissions_11_in_lbs'].notna() & (df['TRI_Air_Emissions_11_in_lbs'] != 0),
        (df['TRI_Air_Emissions_12_in_lbs'] - df['TRI_Air_Emissions_11_in_lbs']) / df['TRI_Air_Emissions_11_in_lbs'],
        np.nan
    )

    df['TRI_Air_Emissions_Growth_Rate_13'] = np.where(
        df['TRI_Air_Emissions_12_in_lbs'].notna() & (df['TRI_Air_Emissions_12_in_lbs'] != 0),
        (df['TRI_Air_Emissions_13_in_lbs'] - df['TRI_Air_Emissions_12_in_lbs']) / df['TRI_Air_Emissions_12_in_lbs'],
        np.nan
    )

    return df

train_df      = create_features1(train_df)
test_df       = create_features1(test_df)
new_features1 = ['TRI_Air_Emissions_YoY_Change_11','TRI_Air_Emissions_YoY_Change_12','TRI_Air_Emissions_YoY_Change_13',
                 'TRI_Air_Emissions_Growth_Rate_11','TRI_Air_Emissions_Growth_Rate_12','TRI_Air_Emissions_Growth_Rate_13']

In [182]:
def create_features2(df):


    # Year-over-year difference
    df['GHG_Direct_Emissions_YoY_Change_11'] = df['GHG_Direct_Emissions_11_in_metric_tons'] - df['GHG_Direct_Emissions_10_in_metric_tons']
    df['GHG_Direct_Emissions_YoY_Change_12'] = df['GHG_Direct_Emissions_12_in_metric_tons'] - df['GHG_Direct_Emissions_11_in_metric_tons']
    df['GHG_Direct_Emissions_YoY_Change_13'] = df['GHG_Direct_Emissions_13_in_metric_tons'] - df['GHG_Direct_Emissions_12_in_metric_tons']

    # Year-over-year growth rate for GHG Direct Emissions with NaN protection
    df['GHG_Direct_Emissions_Growth_Rate_11'] = np.where(
        df['GHG_Direct_Emissions_10_in_metric_tons'].notna() & (df['GHG_Direct_Emissions_10_in_metric_tons'] != 0),
        (df['GHG_Direct_Emissions_11_in_metric_tons'] - df['GHG_Direct_Emissions_10_in_metric_tons']) / df['GHG_Direct_Emissions_10_in_metric_tons'],
        np.nan
    )

    df['GHG_Direct_Emissions_Growth_Rate_12'] = np.where(
        df['GHG_Direct_Emissions_11_in_metric_tons'].notna() & (df['GHG_Direct_Emissions_11_in_metric_tons'] != 0),
        (df['GHG_Direct_Emissions_12_in_metric_tons'] - df['GHG_Direct_Emissions_11_in_metric_tons']) / df['GHG_Direct_Emissions_11_in_metric_tons'],
        np.nan
    )

    df['GHG_Direct_Emissions_Growth_Rate_13'] = np.where(
        df['GHG_Direct_Emissions_12_in_metric_tons'].notna() & (df['GHG_Direct_Emissions_12_in_metric_tons'] != 0),
        (df['GHG_Direct_Emissions_13_in_metric_tons'] - df['GHG_Direct_Emissions_12_in_metric_tons']) / df['GHG_Direct_Emissions_12_in_metric_tons'],
        np.nan
    )

    return df


new_features2 = ['GHG_Direct_Emissions_YoY_Change_11','GHG_Direct_Emissions_YoY_Change_12','GHG_Direct_Emissions_YoY_Change_13',
                 'GHG_Direct_Emissions_Growth_Rate_11','GHG_Direct_Emissions_Growth_Rate_12','GHG_Direct_Emissions_Growth_Rate_13'
                 ]
train_df      = create_features2(train_df)
test_df       = create_features2(test_df)

In [183]:
def create_features3(df):
    df['TRI_to_GHG_Ratio_10'] = np.where(df['GHG_Direct_Emissions_10_in_metric_tons'].notna(),
                                     df['TRI_Air_Emissions_10_in_lbs'] / df['GHG_Direct_Emissions_10_in_metric_tons'],
                                     np.nan)

    df['TRI_to_GHG_Ratio_11'] = np.where(df['GHG_Direct_Emissions_11_in_metric_tons'].notna(),
                                        df['TRI_Air_Emissions_11_in_lbs'] / df['GHG_Direct_Emissions_11_in_metric_tons'],
                                        np.nan)

    df['TRI_to_GHG_Ratio_12'] = np.where(df['GHG_Direct_Emissions_12_in_metric_tons'].notna(),
                                        df['TRI_Air_Emissions_12_in_lbs'] / df['GHG_Direct_Emissions_12_in_metric_tons'],
                                        np.nan)

    df['TRI_to_GHG_Ratio_13'] = np.where(df['GHG_Direct_Emissions_13_in_metric_tons'].notna(),
                                        df['TRI_Air_Emissions_13_in_lbs'] / df['GHG_Direct_Emissions_13_in_metric_tons'],
                                        np.nan)
    return df



train_df      = create_features3(train_df)
test_df       = create_features3(test_df)
new_features3 = ['TRI_to_GHG_Ratio_10','TRI_to_GHG_Ratio_11','TRI_to_GHG_Ratio_12','TRI_to_GHG_Ratio_13']

In [184]:
summary_df = train_df.groupby(['Economic_Sector', 'State']).agg({'GHG_Direct_Emissions_14_in_metric_tons': ['mean', 'median', 'max', 'min','count']})

summary_df.columns = [
                        'GHG_Direct_Emissions_14_in_metric_tons_mean',
                        'GHG_Direct_Emissions_14_in_metric_tons_median',
                        'GHG_Direct_Emissions_14_in_metric_tons_max',
                        'GHG_Direct_Emissions_14_in_metric_tons_min',
                        'GHG_Direct_Emissions_14_in_metric_tons_count'
                     ]

summary_df = summary_df.reset_index()

train_df = train_df.merge(summary_df, on=['Economic_Sector', 'State'], how='left')
test_df  = test_df.merge(summary_df,  on=['Economic_Sector', 'State'], how='left')

In [185]:
test_df.shape

(2508, 41)

In [186]:
train_df.shape

(4655, 42)

In [187]:
merged_df = pd.concat((train_df,test_df),axis=0)
for cols in ['City','State','County','FIPScode','PrimaryNAICS','Economic_Sector']:
    le              = LabelEncoder()
    merged_df[cols] = le.fit_transform(merged_df[cols].values.reshape(-1,1))

train_df = merged_df.iloc[:train_df.shape[0],:]
test_df  = merged_df.iloc[train_df.shape[0]:,:]
train_df.shape,test_df.shape

((4655, 42), (2508, 42))

### ここでもうtest_dfの形が違う

In [188]:
numerical_columns = [
                    'TRI_Air_Emissions_10_in_lbs', 'TRI_Air_Emissions_11_in_lbs',
                    'TRI_Air_Emissions_12_in_lbs', 'TRI_Air_Emissions_13_in_lbs','PrimaryNAICS',
                    'GHG_Direct_Emissions_10_in_metric_tons', 'GHG_Direct_Emissions_11_in_metric_tons',
                    'GHG_Direct_Emissions_12_in_metric_tons', 'GHG_Direct_Emissions_13_in_metric_tons',
                    ]
lat_lon_columns   = ['Latitude','Longitude']
target_columns    = ['GHG_Direct_Emissions_14_in_metric_tons']
categorical_columns = ['City','State','County','FIPScode','PrimaryNAICS','Economic_Sector']
train_aggregations  =  [
                        'GHG_Direct_Emissions_14_in_metric_tons_mean',
                        'GHG_Direct_Emissions_14_in_metric_tons_median',
                        'GHG_Direct_Emissions_14_in_metric_tons_max',
                        'GHG_Direct_Emissions_14_in_metric_tons_min',
                        'GHG_Direct_Emissions_14_in_metric_tons_count'
                     ]
new_features1  = ['TRI_Air_Emissions_YoY_Change_11','TRI_Air_Emissions_YoY_Change_12','TRI_Air_Emissions_YoY_Change_13',
                    'TRI_Air_Emissions_Growth_Rate_11','TRI_Air_Emissions_Growth_Rate_12','TRI_Air_Emissions_Growth_Rate_13']

new_features2   = ['GHG_Direct_Emissions_YoY_Change_11','GHG_Direct_Emissions_YoY_Change_12','GHG_Direct_Emissions_YoY_Change_13',
                  'GHG_Direct_Emissions_Growth_Rate_11','GHG_Direct_Emissions_Growth_Rate_12','GHG_Direct_Emissions_Growth_Rate_13'
                  ]
new_features3   = ['TRI_to_GHG_Ratio_10','TRI_to_GHG_Ratio_11','TRI_to_GHG_Ratio_12','TRI_to_GHG_Ratio_13']
neighbour_feats = ['Economy_Sector_Weighted_Avg','Economic_Sector_Average','Nearest_Weighted_Average','Nearest_Average']


train = train_df[numerical_columns+
                 lat_lon_columns+
                #  categorical_columns+
                 new_features1+['Economic_Sector']
                #  new_features2
               #   new_features3+
                #  train_aggregations+
               #   neighbour_feats
                 ].values
test  = test_df[numerical_columns+
                 lat_lon_columns+
                #  categorical_columns+
                 new_features1+['Economic_Sector']
                #  new_features2
               #   new_features3+
                #  train_aggregations+
               #   neighbour_feats
                 ].values
target = train_df[target_columns].values


In [189]:
len(target)

4655

In [190]:
len(test_df)

2508

In [191]:

def get_models_trained(train,test,target, num_folds):
    kf               = KFold(n_splits=num_folds, shuffle=True, random_state=13)

    oof_predictions  = np.zeros(len(train))
    test_predictions = np.zeros(len(test))


    for fold, (train_index, valid_index) in enumerate(kf.split(train,target)):

        X_train, X_valid             = train[train_index], train[valid_index]
        y_train, y_valid             = target[train_index], target[valid_index]


        model_dict                   = {}
        loss_dict                    = {}
        global valid_pred_dict
        valid_pred_dict              = {}

        params                       = {"n_estimators": 626, "max_depth": 3, "random_state":13,
                                        "min_child_weight": 0.001190123543553736, "learning_rate": 0.010519736270936835,
                                        "subsample": 0.7304788478701394, "colsample_bylevel": 0.604447278915981,
                                          "colsample_bytree": 0.7616852136157319, "reg_alpha": 0.115175569924065, "reg_lambda": 0.07155347824929895}
        model1                       = XGBRegressor(**params)

        params                       = {"n_estimators": 206, "max_depth": 3, "min_child_weight": 0.002124665025111174, "random_state":13,
                                        "learning_rate": 0.02528390491004826, "subsample": 0.800800019181945,
                                        "colsample_bylevel": 0.7264139639244361, "colsample_bytree": 0.7283285945816331,
                                          "reg_alpha": 0.13673920290025274, "reg_lambda": 0.008614256283329808}
        model2                       = XGBRegressor(**params)

        params                       = {"n_estimators": 56,"random_state":13,
                                         "verbose":-1,}
        model3                       = LGBMRegressor(**params)



        _                             = model1.fit(X_train,np.log1p(y_train))
        valid_preds1                  = np.expm1(model1.predict(X_valid))
        rmsle1                        = root_mean_squared_log_error(y_valid, valid_preds1)
        print(f"Fold {fold+1} RMSLE for model1 = {rmsle1}")

        _                             = model2.fit(X_train,np.log1p(y_train))
        valid_preds2                  = np.expm1(model2.predict(X_valid))
        rmsle2                        = root_mean_squared_log_error(y_valid, valid_preds2)
        print(f"Fold {fold+1} RMSLE for model2 = {rmsle2}")

        _                             = model3.fit(X_train,np.log1p(y_train))
        valid_preds3                  = np.expm1(model3.predict(X_valid))
        rmsle3                        = root_mean_squared_log_error(y_valid, valid_preds3)
        print(f"Fold {fold+1} RMSLE for model3 = {rmsle3}")



        loss_dict['model1']           = rmsle1
        loss_dict['model2']           = rmsle2
        loss_dict['model3']           = rmsle3


        model_dict['model1']          = model1
        model_dict['model2']          = model2
        model_dict['model3']          = model3


        valid_pred_dict['model1']     = valid_preds1
        valid_pred_dict['model2']     = valid_preds2
        valid_pred_dict['model3']     = valid_preds3



        valid_preds_mean              = np.mean(list(valid_pred_dict.values()), axis=0)
        rmsle_mean                    = root_mean_squared_log_error(y_valid, valid_preds_mean)
        print(f"Fold {fold+1} Average RMSLE = {rmsle_mean}")


        min_loss_model                = min(loss_dict, key=loss_dict.get)

        model                         = model_dict[min_loss_model]
        valid_preds_best_model        = np.expm1(model.predict(X_valid))
        rmsle_best                    = root_mean_squared_log_error(y_valid, valid_preds_best_model)


        if rmsle_mean >rmsle_best:
            print(f"The average RMSLE is {rmsle_mean} while the best RMSLE is {rmsle_best} and we proceed with the model with best RMSLE")
            model = model_dict[min_loss_model]
            oof_predictions[valid_index]  = valid_preds_best_model
            test_preds                    = model.predict(test)
            test_predictions += (test_preds) / kf.n_splits
            oof_predictions[valid_index]  = valid_preds_best_model


        else:
            print(f"The average RMSLE is {rmsle_mean} while the best RMSLE is {rmsle_best} and we proceed with averaging of all models")
            output_predictions = []
            for key,values in model_dict.items():
                model       = values
                test_preds  = model.predict(test)
                _           = output_predictions.append(test_preds)
            output_preds    = np.mean(output_predictions, axis=0)
            test_predictions += output_preds/kf.n_splits
            oof_predictions[valid_index]  = valid_preds_mean







        gc.collect()

        print('---------------\n')

    RMSLE = root_mean_squared_log_error(target, oof_predictions)
    print(f"OOF RMSLE = {RMSLE}")

    return oof_predictions,np.expm1(test_predictions)

In [192]:
len(test)

2508

In [193]:
oof_predictions,test_predictions = get_models_trained(train,test,target,30)

Fold 1 RMSLE for model1 = 0.6784038334722061
Fold 1 RMSLE for model2 = 0.6731001120759659
Fold 1 RMSLE for model3 = 0.685226641461286
Fold 1 Average RMSLE = 0.675007098850325
The average RMSLE is 0.675007098850325 while the best RMSLE is 0.6731001120759659 and we proceed with the model with best RMSLE
---------------

Fold 2 RMSLE for model1 = 0.7420926269029038
Fold 2 RMSLE for model2 = 0.7441571477231496
Fold 2 RMSLE for model3 = 0.7581723637148129
Fold 2 Average RMSLE = 0.745005693019921
The average RMSLE is 0.745005693019921 while the best RMSLE is 0.7420926269029038 and we proceed with the model with best RMSLE
---------------

Fold 3 RMSLE for model1 = 0.621770042843018
Fold 3 RMSLE for model2 = 0.6243470053436888
Fold 3 RMSLE for model3 = 0.6354542192580105
Fold 3 Average RMSLE = 0.6227495696258979
The average RMSLE is 0.6227495696258979 while the best RMSLE is 0.621770042843018 and we proceed with the model with best RMSLE
---------------

Fold 4 RMSLE for model1 = 0.7216578366

In [194]:
submit = pd.read_csv(path+'sample_submission.csv')
submit

,4655,249574.30382740175
0,4656,249574.303827
1,4657,249574.303827
2,4658,249574.303827
3,4659,249574.303827
4,4660,249574.303827
...,...,...
2502,7158,249574.303827
2503,7159,249574.303827
2504,7160,249574.303827
2505,7161,249574.303827


In [195]:
submit.iloc[:, 1] = test_predictions[:len(submit)]
submit.to_csv( 'submission_2.csv', index = False)


In [196]:
submit

,4655,249574.30382740175
0,4656,44443.223253
1,4657,305030.032498
2,4658,40820.719077
3,4659,26900.439463
4,4660,46264.234377
...,...,...
2502,7158,51299.251884
2503,7159,530373.986000
2504,7160,117380.896125
2505,7161,139032.731234


In [197]:
import time
print(time.ctime())

Mon Nov 18 11:39:04 2024
